In [ ]:
# --- Rush Hour Analysis most recent year

df['time_dt'] = pd.to_datetime(df['time_full_str'])
start_date = pd.Timestamp(f"2024-10-01")
end_date = pd.Timestamp(f"2025-09-30")
all_data = df[(df['time_dt'] <= end_date) & (df['time_dt'] >= start_date) & (df['time_dt'].dt.weekday < 5) & (df['trip_dist'] >= 10) & (df['mode'] == 0)].copy()

hour = all_data['time_dt'].dt.hour
is_rush_hour = ((hour >= 8) & (hour < 10)) | ((hour >= 17) & (hour < 19))
all_data['period'] = 'Non-Rush Hour'
all_data.loc[is_rush_hour, 'period'] = 'Rush Hour'

grouping_cols = ['tripid', 'cityname_corrected', 'lat_orig', 'lon_orig', 'lat_dest', 'lon_dest', 'trip_dist', 'period']
agg_times = all_data.groupby(grouping_cols)['traffic_min'].agg(['median', 'count']).reset_index()

comparison_table = agg_times.pivot_table(
    index=[col for col in grouping_cols if col != 'period'],
    columns='period',
    values=['median', 'count']
).reset_index()

comparison_table.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in comparison_table.columns.values]
comparison_table.rename(columns={
    'median_Non-Rush Hour': 'median_non_rush_hour',
    'median_Rush Hour': 'median_rush_hour',
    'count_Non-Rush Hour': 'count_non_rush_hour',
    'count_Rush Hour': 'count_rush_hour',
    'tripid_': 'tripid',
    'mode_': 'mode',
    'cityname_corrected_': 'cityname_corrected',
    'lat_orig_': 'lat_orig',
    'lon_orig_': 'lon_orig',
    'lat_dest_': 'lat_dest',
    'lon_dest_': 'lon_dest',
    'trip_dist_': 'trip_dist'
}, inplace=True)

comparison_table.fillna(0, inplace=True)
comparison_table["rush_hour_diff"] = comparison_table['median_rush_hour'] - comparison_table['median_non_rush_hour']
comparison_table['rush_hour_impact'] = ((comparison_table['median_rush_hour'] - comparison_table['median_non_rush_hour']) / comparison_table['median_non_rush_hour']) * 100

for col in ['count_non_rush_hour', 'count_rush_hour']:
    if col in comparison_table.columns:
        comparison_table[col] = comparison_table[col].astype(int)

comparison_table['origin'] = comparison_table['lat_orig'].astype(str) + ',' + comparison_table['lon_orig'].astype(str)
comparison_table['dest'] = comparison_table['lat_dest'].astype(str) + ',' + comparison_table['lon_dest'].astype(str)

final_table = comparison_table.sort_values('rush_hour_diff', ascending=False)
display(final_table[(final_table["count_non_rush_hour"] >= 3) & (final_table["count_rush_hour"] >= 3)][['tripid', 'cityname_corrected', 'origin', 'dest', 'trip_dist', 'median_non_rush_hour', 'median_rush_hour', 'rush_hour_impact', 'count_non_rush_hour', 'count_rush_hour']].head(20))


In [ ]:
grouping_cols = ['tripid', 'cityname_corrected', 'lat_orig', 'lon_orig', 'lat_dest', 'lon_dest', 'trip_dist', 'period']
agg_times = all_data.groupby(grouping_cols)['traffic_min'].agg(['median', 'count']).reset_index()

comparison_table = agg_times.pivot_table(
    index=[col for col in grouping_cols if col != 'period'],
    columns='period',
    values=['median', 'count']
).reset_index()

comparison_table.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in comparison_table.columns.values]
comparison_table.rename(columns={
    'median_Non-Rush Hour': 'median_non_rush_hour',
    'median_Rush Hour': 'median_rush_hour',
    'count_Non-Rush Hour': 'count_non_rush_hour',
    'count_Rush Hour': 'count_rush_hour',
    'tripid_': 'tripid',
    'mode_': 'mode',
    'cityname_corrected_': 'cityname_corrected',
    'lat_orig_': 'lat_orig',
    'lon_orig_': 'lon_orig',
    'lat_dest_': 'lat_dest',
    'lon_dest_': 'lon_dest',
    'trip_dist_': 'trip_dist'
}, inplace=True)

comparison_table.fillna(0, inplace=True)
comparison_table["rush_hour_diff"] = comparison_table['median_rush_hour'] - comparison_table['median_non_rush_hour']
comparison_table['rush_hour_impact'] = ((comparison_table['median_rush_hour'] - comparison_table['median_non_rush_hour']) / comparison_table['median_non_rush_hour']) * 100

for col in ['count_non_rush_hour', 'count_rush_hour']:
    if col in comparison_table.columns:
        comparison_table[col] = comparison_table[col].astype(int)

comparison_table['origin'] = comparison_table['lat_orig'].astype(str) + ',' + comparison_table['lon_orig'].astype(str)
comparison_table['dest'] = comparison_table['lat_dest'].astype(str) + ',' + comparison_table['lon_dest'].astype(str)

final_table = comparison_table.sort_values('rush_hour_diff', ascending=False)
display(final_table[(final_table["count_non_rush_hour"] >= 3) & (final_table["count_rush_hour"] >= 3)][['tripid', 'cityname_corrected', 'origin', 'dest', 'trip_dist', 'median_non_rush_hour', 'median_rush_hour', 'rush_hour_impact', 'count_non_rush_hour', 'count_rush_hour']].head(20))


In [ ]:
# Convert to float32 to save memory
all_data['lat_orig'] = all_data['lat_orig'].astype(np.float32)
all_data['lon_orig'] = all_data['lon_orig'].astype(np.float32)
all_data['lat_dest'] = all_data['lat_dest'].astype(np.float32)
all_data['lon_dest'] = all_data['lon_dest'].astype(np.float32)

def cluster_and_assign(full_coords, sample_size=10000, eps_km=0.5):
    # Sample for clustering
    if len(full_coords) > sample_size:
        sample_idx = np.random.choice(len(full_coords), sample_size, replace=False)
        sample_coords = full_coords[sample_idx]
    else:
        sample_idx = np.arange(len(full_coords))
        sample_coords = full_coords

    # Cluster the sample
    db = DBSCAN(eps=eps_km/6371, min_samples=1, metric='haversine', algorithm='ball_tree')
    db.fit(np.radians(sample_coords))
    sample_labels = db.labels_

    # Assign clusters to all points using nearest neighbor in the sample
    tree = BallTree(np.radians(sample_coords), metric='haversine')
    dist, ind = tree.query(np.radians(full_coords), k=1)
    assigned_labels = sample_labels[ind.flatten()]
    return assigned_labels

# Cluster origins
orig_coords = all_data[['lat_orig', 'lon_orig']].values
all_data['origin_cluster'] = cluster_and_assign(orig_coords)

# Cluster destinations
dest_coords = all_data[['lat_dest', 'lon_dest']].values
all_data['dest_cluster'] = cluster_and_assign(dest_coords)

# Combine both clusters to define a route group
all_data['route_group'] = all_data['origin_cluster'].astype(str) + '_' + all_data['dest_cluster'].astype(str)

all_data.head()

In [ ]:
# --- Rush Hour Analysis by Route Group (without trip_dist in grouping) ---
# This cell groups by route_group and computes rush hour/non-rush hour comparison as before, then merges trip_dist back in

grouping_cols = ['route_group', 'cityname_corrected', 'period']

# Ensure 'period' column exists
if 'period' not in all_data.columns:
    hour = all_data['time_dt'].dt.hour
    is_rush_hour = ((hour >= 8) & (hour < 10)) | ((hour >= 17) & (hour < 19))
    all_data['period'] = 'Non-Rush Hour'
    all_data.loc[is_rush_hour, 'period'] = 'Rush Hour'

agg_times = all_data.groupby(grouping_cols)['traffic_min'].agg(['mean', 'count']).reset_index()

comparison_table = agg_times.pivot_table(
    index=[col for col in grouping_cols if col != 'period'],
    columns='period',
    values=['mean', 'count']
).reset_index()

comparison_table.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in comparison_table.columns.values]
comparison_table.rename(columns={
    'mean_Non-Rush Hour': 'mean_non_rush_hour',
    'mean_Rush Hour': 'mean_rush_hour',
    'count_Non-Rush Hour': 'count_non_rush_hour',
    'count_Rush Hour': 'count_rush_hour',
    'route_group_': 'route_group',
    'cityname_corrected_': 'cityname_corrected'
}, inplace=True)

comparison_table.fillna(0, inplace=True)
comparison_table["rush_hour_diff"] = comparison_table['mean_rush_hour'] - comparison_table['mean_non_rush_hour']
# comparison_table['rush_hour_impact'] = ((comparison_table['mean_rush_hour'] - comparison_table['mean_non_rush_hour']) / comparison_table['mean_non_rush_hour']) * 100

for col in ['count_non_rush_hour', 'count_rush_hour']:
    if col in comparison_table.columns:
        comparison_table[col] = comparison_table[col].astype(int)

# Merge trip_dist back in (using first value for each route_group)
trip_dist_map = all_data.groupby('route_group')[['trip_dist', "origin", "dest"]].first().reset_index()
comparison_table = comparison_table.merge(trip_dist_map, on='route_group', how='left')

# Show top 20 grouped routes by rush hour impact
comparison_table = comparison_table.sort_values('rush_hour_diff', ascending=False)
display_table = comparison_table[(comparison_table["count_non_rush_hour"] >= 20) & (comparison_table["count_rush_hour"] >= 15)][['route_group', 'cityname_corrected', 'trip_dist', "origin", "dest", 'mean_non_rush_hour', 'median_rush_hour', 'count_non_rush_hour', 'count_rush_hour', "rush_hour_diff"]]
display_table.head(10)

In [ ]:
# Save top-3 route maps per city into ./plots_by_route
import pathlib
import re
import matplotlib.pyplot as plt
import geopandas as gpd

out_dir = pathlib.Path('plots_by_route')
out_dir.mkdir(parents=True, exist_ok=True)

# Reconstruct line_gdf/poly_gdf from results if needed
if 'line_gdf' not in globals() and 'results' in globals() and results:
    line_gdf = gpd.GeoDataFrame(results, geometry='linestring', crs='EPSG:4326')
    line_gdf = line_gdf.set_geometry('linestring')
if 'poly_gdf' not in globals() and 'line_gdf' in globals():
    line_gdf_web = line_gdf.to_crs(epsg=3857)
    line_gdf_web['polygon_100m'] = line_gdf_web.geometry.buffer(100)
    poly_gdf = gpd.GeoDataFrame(line_gdf_web.copy(), geometry=line_gdf_web['polygon_100m'], crs=line_gdf_web.crs).to_crs(epsg=4326)

def safe(s):
    if s is None:
        return 'unknown'
    s = str(s)
    s = re.sub(r'[^A-Za-z0-9_\-]', '_', s)
    return s[:180]

if 'final_table' not in globals():
    raise RuntimeError('`final_table` not found. Run the Rush Hour Analysis cell first.')

ft = final_table.reset_index()
if 'rank' not in ft.columns:
    ft.insert(0, 'rank', range(1, len(ft) + 1))

for city in ft['cityname_corrected'].unique():
    city_rows = ft[ft['cityname_corrected'] == city].sort_values('rush_vs_freeflow_diff', ascending=False).head(3)
    if city_rows.empty:
        continue
    for _, crow in city_rows.iterrows():
        rank = int(crow.get('rank', -1))
        tripid = crow.get('tripid')
        match = None
        if 'line_gdf' in globals():
            try:
                if 'tripid' in line_gdf.columns:
                    match = line_gdf[line_gdf['tripid'] == tripid]
                    if match.empty:
                        match = None
                else:
                    match = None
            except Exception:
                match = None
        if match is None:
            try:
                match = line_gdf.iloc[[0]] if 'line_gdf' in globals() and len(line_gdf) > 0 else None
            except Exception:
                match = None

        if match is None or match.empty:
            print(f'No geometry found for trip {tripid} (rank {rank}) in city {city}; skipping.')
            continue

        fname = out_dir / f"{rank:03d}_{safe(tripid)}_{safe(city)}.png"

        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        try:
            if 'poly_gdf' in globals():
                try:
                    single_poly = poly_gdf[poly_gdf['tripid'] == tripid] if 'tripid' in poly_gdf.columns else poly_gdf.iloc[[match.index[0]]]
                except Exception:
                    single_poly = poly_gdf.iloc[[match.index[0]]] if len(poly_gdf) > match.index[0] else None
                if single_poly is not None and not single_poly.empty:
                    single_poly.to_crs(epsg=3857).plot(ax=ax, color='orange', alpha=0.4, edgecolor='darkorange')
            match.to_crs(epsg=3857).plot(ax=ax, color='red', linewidth=3)
            minx, miny, maxx, maxy = match.to_crs(epsg=3857).total_bounds
            buf = max((maxx - minx), (maxy - miny)) * 0.25 if maxx > minx and maxy > miny else 200
            ax.set_xlim(minx - buf, maxx + buf)
            ax.set_ylim(miny - buf, maxy + buf)
            try:
                import contextily as ctx
                ctx.add_basemap(ax)
            except Exception:
                pass
            ax.set_axis_off()
            ax.set_title(f'Rank {rank} — Trip {tripid} — {city}')
            fig.savefig(fname, dpi=150, bbox_inches='tight')
            plt.close(fig)
            print(f'Saved {fname}')
        except Exception as e:
            print(f'Failed to save route {tripid} (rank {rank}) for city {city}: {e}')


Backup

start_date = pd.Timestamp(f"2024-10-01")
end_date = pd.Timestamp(f"2025-09-30")

df = df[(df['time_dt'] <= end_date) & (df['time_dt'] >= start_date)]

display(df["time_dt"].min(), df["time_dt"].max())

Let's now see how many cities we have in this dataset.

In [ ]:
df.loc[:, ["cityname_corrected", "tz"]].drop_duplicates()

,cityname_corrected,tz
3970387,Calgary,America/Edmonton
3971133,Edmonton,America/Edmonton
3971843,Halifax,America/Halifax
3972826,Hamilton,America/Toronto
3973695,Kitchener-Cambrigde-Waterloo,America/Toronto
3974686,London,America/Toronto
3975805,Montréal,America/Toronto
3977878,Oshawa,America/Toronto
3978757,Ottawa,America/Toronto
3980304,Québec,America/Toronto


For this table, I’ve excluded routes that are less than 10km long, and only included routes with at least 3 data points collected over the past 6 months in both non-rush hour and rush hour (8-10am, 5-7pm) times. I've excluded everything under 10km, because that doesn't really feel like it qualifies as a commute.

In [ ]:
# --- Rush Hour Analysis most recent year
weekdays_df = df[(df['time_dt'].dt.weekday < 5) & (df['trip_dist'] >= MIN_TRIP_DISTANCE) & (df['mode'] == 0)].copy()

hour = weekdays_df['time_dt'].dt.hour

is_rush_hour = ((hour >= MORNING_RUSH_HOUR_START) & (hour < MORNING_RUSH_HOUR_END)) | ((hour >= EVENING_RUSH_HOUR_START) & (hour < EVENING_RUSH_HOUR_END))

weekdays_df['period'] = 'Non-Rush Hour'
weekdays_df.loc[is_rush_hour, 'period'] = 'Rush Hour'

In [ ]:
# --- Rush Hour Analysis most recent year

# Ensure origin/dest exist (from earlier cells)
if 'origin' not in weekdays_df.columns and 'lat_orig' in weekdays_df.columns:
    weekdays_df['origin'] = weekdays_df['lat_orig'].astype(str) + ',' + weekdays_df['lon_orig'].astype(str)
if 'dest' not in weekdays_df.columns and 'lat_dest' in weekdays_df.columns:
    weekdays_df['dest'] = weekdays_df['lat_dest'].astype(str) + ',' + weekdays_df['lon_dest'].astype(str)

# Group traffic_min by trip/period to get mean and count per period
grouping_cols = ['tripid', 'cityname_corrected', 'trip_dist', 'period']
agg_times = weekdays_df.groupby(grouping_cols)['traffic_min'].agg(['mean', 'count']).reset_index()

# Pivot traffic stats so we have mean/count per period side-by-side
comparison_table = agg_times.pivot_table(
    index=[col for col in grouping_cols if col != 'period'],
    columns='period',
    values=['mean', 'count']
).reset_index()

# Flatten multiindex column names
comparison_table.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in comparison_table.columns.values]

# Rename traffic columns to friendly names
comparison_table.rename(columns={
    'mean_Non-Rush Hour': 'mean_non_rush_hour',
    'mean_Rush Hour': 'mean_rush_hour',
    'count_Non-Rush Hour': 'count_non_rush_hour',
    'count_Rush Hour': 'count_rush_hour',
    'tripid_': 'tripid',
    'mode_': 'mode',
    'cityname_corrected_': 'cityname_corrected',
    'trip_dist_': 'trip_dist',
}, inplace=True)

# Compute a single overall mean for notraffic_min (include all periods)
notraffic_overall = weekdays_df.groupby(['tripid', 'cityname_corrected', 'trip_dist'])['notraffic_min'].mean().reset_index().rename(columns={'notraffic_min': 'mean_notraffic'})

# Compute 90th percentile travel time during Rush Hour per trip (p90)
rush_df = weekdays_df[weekdays_df['period'] == 'Rush Hour']
p90_rush = rush_df.groupby(['tripid', 'cityname_corrected', 'trip_dist'])['traffic_min'].quantile(0.9).reset_index().rename(columns={'traffic_min': 'p90_rush_hour'})

# Merge the overall notraffic mean and p90 into the comparison table
comparison_table = comparison_table.merge(notraffic_overall, on=['tripid', 'cityname_corrected', 'trip_dist'], how='left')
comparison_table = comparison_table.merge(p90_rush, on=['tripid', 'cityname_corrected', 'trip_dist'], how='left')

# Bring origin/dest into the comparison table using a representative value (first)
orig_dest = weekdays_df.groupby(['tripid', 'cityname_corrected', 'trip_dist']).agg({'origin': 'first', 'dest': 'first'}).reset_index()
comparison_table = comparison_table.merge(orig_dest, on=['tripid', 'cityname_corrected', 'trip_dist'], how='left')

comparison_table.fillna(0, inplace=True)
# New metric: difference between rush hour mean and overall notraffic (free-flow)
comparison_table['rush_vs_freeflow_diff'] = comparison_table['mean_rush_hour'] - comparison_table['mean_notraffic']
# Percent impact relative to free-flow (notraffic)
# Avoid division by zero
comparison_table['rush_vs_freeflow_pct'] = comparison_table.apply(lambda r: (r['rush_vs_freeflow_diff'] / r['mean_notraffic'] * 100) if r['mean_notraffic'] else 0, axis=1)

# Convert count columns to integers where present
for col in ['count_non_rush_hour', 'count_rush_hour']:
    if col in comparison_table.columns:
        comparison_table[col] = comparison_table[col].astype(int)

# Round numeric columns: trip_dist to 1 decimal, others to 2 decimals
if 'trip_dist' in comparison_table.columns:
    comparison_table['trip_dist'] = comparison_table['trip_dist'].round(1)
num_cols = ['mean_non_rush_hour', 'mean_rush_hour', 'mean_notraffic', 'rush_vs_freeflow_diff', 'rush_vs_freeflow_pct', 'p90_rush_hour']
for c in num_cols:
    if c in comparison_table.columns:
        comparison_table[c] = comparison_table[c].round(2)

# Final sorting and filtering
final_table = comparison_table.sort_values('rush_vs_freeflow_diff', ascending=False)
final_table = final_table[(final_table['count_non_rush_hour'] >= MIN_DATA_POINTS_NONRUSHHOUR) & (final_table['count_rush_hour'] >= MIN_DATA_POINTS_RUSHHOUR)]

# Include origin/dest and p90 in display
final_table["rank"] = range(1,len(final_table)+1)
display_cols = ['tripid', 'cityname_corrected', 'origin', 'dest', 'trip_dist', 'mean_notraffic', 'mean_rush_hour', 'rush_vs_freeflow_diff', 'rush_vs_freeflow_pct', 'p90_rush_hour']
final_table = final_table.set_index('rank')
final_table[display_cols].head(10)

,tripid,cityname_corrected,origin,dest,trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct,p90_rush_hour
rank,,,,,,,,,,
1,101617601,Toronto,"43.867138,-79.50793","43.65489,-79.385704",43.0,35.78,62.09,26.31,73.53,69.35
2,101610311,Toronto,"43.821083,-79.34588","43.652157,-79.380554",22.8,18.95,40.18,21.22,111.97,51.27
3,303928020,Montréal,"45.50147,-73.46785","45.694843,-73.66869",38.5,31.76,51.82,20.06,63.17,67.24
4,304101661,Toronto,"43.401985,-79.76679","43.648182,-79.45746",41.1,26.67,45.53,18.86,70.71,54.93
5,101636931,Vancouver,"49.299095,-122.79921","49.240627,-123.11095",33.2,35.46,52.34,16.88,47.59,55.80
6,304156470,Vancouver,"49.11764,-122.86204","49.22628,-122.941",19.8,21.22,36.23,15.01,70.71,39.31
7,14182640,Vancouver,"49.25273,-123.11198","49.236145,-122.90907",18.7,21.92,36.14,14.22,64.86,51.09
8,431073181,Montréal,"45.53017,-73.60811","45.704674,-73.618225",29.5,27.10,40.76,13.67,50.43,54.03
9,303925231,Montréal,"45.483418,-73.721565","45.472343,-73.61273",10.6,9.62,23.25,13.63,141.72,26.88


## Individual city data

In [ ]:
for city in final_table["cityname_corrected"].unique():
    city_table = final_table.loc[final_table["cityname_corrected"] == city, ["tripid", "origin", "dest", "trip_dist", "mean_notraffic", "mean_rush_hour", "rush_vs_freeflow_diff", "rush_vs_freeflow_pct"]]
    print(f"Top 5 results for {city}.")
    display(city_table.head(3))

Top 5 results for Toronto.


,tripid,origin,dest,trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct
rank,,,,,,,,
1,101617601,"43.867138,-79.50793","43.65489,-79.385704",43.0,35.78,62.09,26.31,73.53
2,101610311,"43.821083,-79.34588","43.652157,-79.380554",22.8,18.95,40.18,21.22,111.97
4,304101661,"43.401985,-79.76679","43.648182,-79.45746",41.1,26.67,45.53,18.86,70.71


Top 5 results for Montréal.


,tripid,origin,dest,trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct
rank,,,,,,,,
3,303928020,"45.50147,-73.46785","45.694843,-73.66869",38.5,31.76,51.82,20.06,63.17
8,431073181,"45.53017,-73.60811","45.704674,-73.618225",29.5,27.10,40.76,13.67,50.43
9,303925231,"45.483418,-73.721565","45.472343,-73.61273",10.6,9.62,23.25,13.63,141.72


Top 5 results for Vancouver.


...

,tripid,origin,dest,trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct
rank,,,,,,,,
313,13242571,"43.514378,-80.50356","43.42468,-80.51249",18.5,14.94,18.46,3.53,23.62
319,303882720,"43.481506,-80.54236","43.426426,-80.4912",11.0,13.87,17.35,3.48,25.06
351,13242601,"43.510643,-80.4991","43.422935,-80.520386",17.8,14.14,17.46,3.32,23.48


Top 5 results for St. Catharines-Niagara.


,tripid,origin,dest,trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct
rank,,,,,,,,
318,304061570,"43.196846,-79.21198","43.131237,-79.24769",10.1,14.30,17.78,3.48,24.31
423,304061541,"43.177822,-79.24254","43.113194,-79.20374",10.6,11.81,14.79,2.98,25.25
427,101599561,"43.10041,-79.20065","43.166557,-79.24872",15.0,13.93,16.89,2.96,21.26


Top 5 results for Halifax.


,tripid,origin,dest,trip_dist,mean_notraffic,mean_rush_hour,rush_vs_freeflow_diff,rush_vs_freeflow_pct
rank,,,,,,,,
328,101522241,"44.6554,-63.486427","44.653446,-63.59114",17.3,19.46,22.90,3.43,17.64
584,303855790,"44.65711,-63.640923","44.66712,-63.557495",12.8,15.32,17.65,2.33,15.19
601,101517511,"44.691776,-63.59114","44.641476,-63.585648",10.9,15.53,17.81,2.28,14.67


In [ ]:
# Save top-3 route maps per city and fetch fresh geometry from Google Directions for each origin/destination
import os
import pathlib
import re
import time
import requests
import polyline
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import LineString

out_dir = pathlib.Path('plots_by_route')
out_dir.mkdir(parents=True, exist_ok=True)

# Read API key (file fallback then env)
API_KEY = None
key_path = os.path.join(os.getcwd(), 'google_maps_api_key.txt')
if os.path.exists(key_path):
    with open(key_path, 'r', encoding='utf-8') as fh:
        API_KEY = fh.read().strip()
if not API_KEY:
    API_KEY = os.getenv('GOOGLE_API_KEY')
if not API_KEY:
    raise RuntimeError('Google Maps API key not found. Add google_maps_api_key.txt or set GOOGLE_API_KEY')

def safe(s):
    if s is None:
        return 'unknown'
    s = str(s)
    s = re.sub(r'[^A-Za-z0-9_\-]', '_', s)
    return s[:180]

if 'final_table' not in globals():
    raise RuntimeError('`final_table` not found. Run the Rush Hour Analysis cell first.')

ft = final_table.reset_index()
if 'rank' not in ft.columns:
    ft.insert(0, 'rank', range(1, len(ft) + 1))

# cache to avoid duplicate API calls for identical origin/dest within this run
route_cache = {}

def fetch_route_geometry(origin_str, dest_str, max_retries=3):
    key = f'{origin_str}|{dest_str}'
    if key in route_cache:
        return route_cache[key]
    url = 'https://maps.googleapis.com/maps/api/directions/json'
    params = {
        'origin': origin_str,
        'destination': dest_str,
        'alternatives': 'false',
        'mode': 'driving',
        'key': API_KEY
    }
    backoff = 1.0
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, params=params, timeout=15)
            resp.raise_for_status()
            j = resp.json()
        except Exception as e:
            if attempt == max_retries:
                raise
            time.sleep(backoff)
            backoff *= 2
            continue
        status = j.get('status')
        if status != 'OK' or not j.get('routes'):
            # For non-retryable API errors, return None
            return None
        ov = j['routes'][0].get('overview_polyline', {}).get('points')
        if not ov:
            return None
        coords_latlon = polyline.decode(ov)  # list of (lat, lon)
        coords = [(lon, lat) for lat, lon in coords_latlon]
        ls = LineString(coords)
        # create polygon buffer of 100 m by projecting to web mercator
        single_line_gdf = gpd.GeoDataFrame([{'tripid': None, 'origin': origin_str, 'dest': dest_str, 'linestring': ls}], geometry='linestring', crs='EPSG:4326')
        web = single_line_gdf.to_crs(epsg=3857)
        web['polygon_100m'] = web.geometry.buffer(100)
        poly = gpd.GeoDataFrame(web.copy(), geometry=web['polygon_100m'], crs=web.crs).to_crs(epsg=4326)
        route_cache[key] = (ls, poly)
        return ls, poly
    return None

for city in ft['cityname_corrected'].unique():
    city_rows = ft[ft['cityname_corrected'] == city].sort_values('rush_vs_freeflow_diff', ascending=False).head(3)
    if city_rows.empty:
        continue
    for _, crow in city_rows.iterrows():
        rank = int(crow.get('rank', -1))
        tripid = crow.get('tripid')
        origin_val = crow.get('origin')
        dest_val = crow.get('dest')
        if not origin_val or not dest_val:
            print(f'Missing origin/dest for trip {tripid} (rank {rank}) in {city}; skipping')
            continue
        # origin_val/dest_val should be 'lat,lon' strings; ensure trimmed
        origin_str = str(origin_val).strip()
        dest_str = str(dest_val).strip()
        try:
            res = fetch_route_geometry(origin_str, dest_str)
        except Exception as e:
            print(f'Error fetching route for {tripid} in {city}: {e}; skipping')
            continue
        if not res:
            print(f'No route geometry returned for {tripid} in {city}; skipping')
            continue
        ls, poly = res
        fname = out_dir / f"{rank:03d}_{safe(tripid)}_{safe(city)}.png"
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        try:
            if poly is not None and not poly.empty:
                poly.to_crs(epsg=3857).plot(ax=ax, color='orange', alpha=0.4, edgecolor='darkorange')
            single_line = gpd.GeoDataFrame([{'linestring': ls}], geometry='linestring', crs='EPSG:4326')
            single_line.to_crs(epsg=3857).plot(ax=ax, color='red', linewidth=3)
            minx, miny, maxx, maxy = single_line.to_crs(epsg=3857).total_bounds
            buf = max((maxx - minx), (maxy - miny)) * 0.25 if maxx > minx and maxy > miny else 200
            ax.set_xlim(minx - buf, maxx + buf)
            ax.set_ylim(miny - buf, maxy + buf)
            try:
                import contextily as ctx
                ctx.add_basemap(ax)
            except Exception:
                pass
            ax.set_axis_off()
            ax.set_title(f'Rank {rank} — Trip {tripid} — {city}')
            fig.savefig(fname, dpi=150, bbox_inches='tight')
            plt.close(fig)
            print(f'Saved {fname}')
        except Exception as e:
            print(f'Failed to save route {tripid} (rank {rank}) for city {city}: {e}')


Saved plots_by_route\001_101617601_Toronto.png
Saved plots_by_route\002_101610311_Toronto.png
Saved plots_by_route\002_101610311_Toronto.png
Saved plots_by_route\004_304101661_Toronto.png
Saved plots_by_route\004_304101661_Toronto.png
Saved plots_by_route\003_303928020_Montr_al.png
Saved plots_by_route\003_303928020_Montr_al.png
Saved plots_by_route\008_431073181_Montr_al.png
Saved plots_by_route\008_431073181_Montr_al.png
Saved plots_by_route\009_303925231_Montr_al.png
Saved plots_by_route\009_303925231_Montr_al.png
